In [ ]:

from classes.event_graph import EventGraph
from classes.event_node import EventNode
from data.event_types import event_types

n0 = EventNode(event_types[0]) 
n1 = EventNode(event_types[1]) 
n2 = EventNode(event_types[2])

print("----------------------------")

eg = EventGraph()
eg.add_node(n0)
eg.add_node(n1)
eg.add_node(n2)
eg.add_edge(n0, n1)
eg.add_edge(n1, n2)
eg.visualize()

In [ ]:
import hashlib

def stable_hash(label: str):
    h = hashlib.blake2b(label.encode("utf-8"), digest_size=16).digest()
    return int.from_bytes(h, "little")

def WL_neighborhood_labels(graph: EventGraph, iterations: int = 3, inspect: bool = False):
    labels = {node: stable_hash(graph.nodes[node].event_type) for node in graph.nodes}
    history = [labels.copy()]
    
    for _ in range(iterations):
        new_labels = {}
        for node in graph.nodes:
            nbrs = graph.adj_list.get(node, [])  # safe default
            neighbor_labels = [labels[nbr] for nbr in nbrs if nbr in labels]
            
            # deterministic sort even if labels later become non-strings
            neighbor_labels.sort(key=repr)

            signature = (labels[node], tuple(neighbor_labels))

            if inspect:
                # keep inspection output readable and still a string so next iter is safe
                new_label = signature
            else:
                new_label = stable_hash(repr(signature))
            
            new_labels[node] = new_label
            
        labels = new_labels
        history.append(labels.copy())
        
    return history  # list of dicts for it=0..h

In [ ]:
labels = WL_neighborhood_labels(eg)
print(labels)

In [ ]:
def graph_to_fingerprint(graph, D=1024):
    all_labels = WL_neighborhood_labels(graph)
    fingerprint = [0.0] * D
    for labels in all_labels:          # use 0..h
        for feat in labels.values():
            idx = feat % D
            fingerprint[idx] = 1.0       # or +=1 for counts
    return fingerprint

In [ ]:
fingerprint = graph_to_fingerprint(eg)

In [ ]:
from lib.pinecone import vdb

print(fingerprint)

vdb.upsert(
  vectors=[
    {
      "id": "A", 
      "values": fingerprint,
      "metadata": { 'country': 'finland '}
    },
    {
      "id": "B", 
      "values": fingerprint,
      "metadata": { 'country': 'finland '}
    }
  ],
  namespace="wwmp"
)

